In [1]:

import os
from typing import List, Dict, Any
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_community.tools import TavilySearchResults
from langchain_community.tools import WikipediaQueryRun
from dotenv import load_dotenv
import wikipedia
import warnings
warnings.filterwarnings('ignore')


c:\Users\winds\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.13\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")


In [11]:
# 카페 메뉴 데이터 로드
def load_cafe_menu_data():
    """카페 메뉴 데이터를 로드하고 Document 객체로 변환"""
    loader = TextLoader("../../data/cafe_menu_data.txt", encoding="utf-8")
    documents = loader.load()
    
    # 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(documents)
    
    return splits

# 벡터 DB 구축
def build_vector_db():
    documents = load_cafe_menu_data()
    
    print(f"총 {len(documents)}개의 문서 청크 생성")
    
    # 임베딩 모델 초기화
    embeddings = OpenAIEmbeddings()
    
    
    # FAISS 벡터 스토어 생성
    vectorstore = FAISS.from_documents(documents, embeddings)
    
    # 벡터 DB 저장
    db_path = "../db/cafe_db"
    vectorstore.save_local(db_path)
    print(f"벡터 DB가 {db_path}에 저장되었습니다.")
    
    return vectorstore

In [12]:
vectorstore = build_vector_db()


총 3개의 문서 청크 생성
벡터 DB가 ../db/cafe_db에 저장되었습니다.


In [5]:

tavily_search_tool = TavilySearchResults(
    api_key=TAVILY_API_KEY,
    max_results=3
)


@tool
def wiki_summary(topic: str) -> str:
    """ 
    args:
        topic : str
    return: 
        answer : str
    """
    try:
        wikipedia.set_lang("ko")
        
        page = wikipedia.page(topic)
        
        summary = page.summary[:500] + "..." if len(page.summary) > 500 else page.summary
        
        return f"주제: {topic}\n요약: {summary}\nURL: {page.url}"
    except wikipedia.exceptions.DisambiguationError as e:
        try:
            page = wikipedia.page(e.options[0])
            summary = page.summary[:500] + "..." if len(page.summary) > 500 else page.summary
            return f"주제: {e.options[0]}\n요약: {summary}\n URL: {page.url}"
        except:
            return f"'{topic}'에 대한 정보 없음"
    except Exception as e:
        return f"위키피디아 검색 중 오류가 발생했습니다: {str(e)}"

@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """
    args:
        query : str
    """
    try:
        docs = vectorstore.similarity_search(query, k=3)
        return docs
    except Exception as e:
        print(f"DB 검색 중 오류가 발생했습니다: {str(e)}")
        return []


tools = [tavily_search_tool, wiki_summary, db_search_cafe_func]

for tool in tools:
    print(f"- {tool.name}: {tool.description}")


- tavily_search_results_json: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.
- wiki_summary: args:
    topic : str
return: 
    answer : str
- db_search_cafe_func: args:
    query : str


In [6]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
)


llm_with_tools = llm.bind_tools(tools)


print(f"사용 가능한 도구: {[tool.name for tool in tools]}")


사용 가능한 도구: ['tavily_search_results_json', 'wiki_summary', 'db_search_cafe_func']


In [7]:
@chain
def tool_calling_chain(user_question: str) -> str:
    """ 도구 호출 체인 """
    response = llm_with_tools.invoke(user_question)
    
    if hasattr(response, 'tool_calls') and response.tool_calls:
        tool_results = []

        for tool_call in response.tool_calls:
        
            tool_name = tool_call['name']
            tool_args = tool_call['args']
            tool_id = tool_call['id']
            

            if tool_name == "tavily_search_results_json":
                result = tavily_search_tool.invoke(tool_args)
            elif tool_name == "wiki_summary":
                result = wiki_summary.invoke(tool_args)
            elif tool_name == "db_search_cafe_func":
                docs = db_search_cafe_func.invoke(tool_args)
                result = "\n".join([doc.page_content for doc in docs])
            else:
                result = f"이상한 도구? : {tool_name}"
            
            tool_results.append(f"도구 {tool_name} 결과:\n{result}")
        
        final_prompt = ChatPromptTemplate.from_template(
            """사용자 질문: {question}
            도구 실행 결과:
            {tool_results}

            위 정보를 바탕으로 사용자 질문에 대한 답변을 제공해주세요.
            답변은 한국어로 작성하고, 구체적인 정보(가격, 재료 등)를 포함해주세요."""
        )
        
        final_chain = final_prompt | llm | StrOutputParser()
        final_answer = final_chain.invoke({
            "question": user_question,
            "tool_results": "\n\n".join(tool_results)
        })
        
        return final_answer
    
    else:
        # 도구 호출이 없는 경우 기본 응답
        return "질문에 대답 못하겠습니다. 구체적으로 적어주세요"



In [8]:
test_question = "아메리카노의 가격과 특징은 무엇인가요?"

answer = tool_calling_chain.invoke(test_question)
print(f"답변: {answer}")

답변: 아메리카노는 에스프레소를 뜨거운 물로 희석하여 만든 커피 음료로, 가격은 보통 ₩4,500 정도입니다. 주요 원료는 에스프레소와 뜨거운 물이며, 설탕이나 시럽을 추가하여 맛을 조절할 수 있습니다. 아메리카노는 클래식한 블랙 커피로, 원두 본연의 맛을 가장 잘 느낄 수 있으며 깔끔하고 깊은 풍미가 특징입니다. 더운 날씨에는 시원하게 즐기기 좋은 음료입니다.


In [9]:
test_question2 = "카페라떼와 카푸치노의 차이점과 각각의 가격을 알려주세요."

answer2 = tool_calling_chain.invoke(test_question2)
print(f"답변: {answer2}")

답변: 카페라떼와 카푸치노의 차이는 주로 우유와 에스프레소의 비율, 우유 거품의 두께, 그리고 마무리로 라테 아트를 만들어내는 것에 있습니다. 카페라떼는 우유의 양이 많고 우유 거품의 두께가 얇아 부드러운 질감을 가지며, 카푸치노는 우유의 양이 적고 거품의 양이 많아 진한 에스프레소의 맛을 느낄 수 있습니다.

일반적으로 카페라떼의 가격은 카푸치노보다 조금 더 비싸며, 가격은 매장이나 지역에 따라 다를 수 있습니다. 일반적으로 카페라떼는 4,000원에서 6,000원 사이에 판매되고, 카푸치노는 3,500원에서 5,500원 사이에 판매될 수 있습니다. 따라서 카페라떼가 카푸치노보다 조금 더 비싼 편이라고 할 수 있습니다.


In [10]:
test_question = "MSM과 콘드로이친의 차이는?"

answer = tool_calling_chain.invoke(test_question)
print(f"답변: {answer}")

답변: MSM과 콘드로이친은 서로 다른 물질입니다.

MSM은 메틸설폰일메탄(Methylsulfonylmethane)의 약자로, 유기황 화합물로서 관절 건강에 도움을 줄 수 있는 건강 보조제로 사용됩니다. 관절염이나 연골 손상으로 인한 통증을 완화하거나 관절 건강을 유지하는 데 도움을 줄 수 있습니다. MSM은 식품으로 섭취할 수 있으며, 건강 보조제로 판매되고 있습니다.

반면, 콘드로이친은 구립운석(chondrite)이라고도 불리는 석질운석의 일종으로, 운석의 원석이 용융이나 분화로 변경되지 않은 것을 말합니다. 콘드로이친은 우주에서 형성된 미세한 입자들이 모여서 형성된 운석으로, 지구상에서 발견된 운석의 대부분이 콘드로이친입니다.

따라서, MSM은 건강 보조제로 사용되는 화합물이고, 콘드로이친은 운석의 일종으로 지구상에서 발견되는 미네랄입니다.
